# Script SAE15 - Traiter des données




---


Nom :  Azzouzi

Prénom : Oussama Abderraouf

Groupe de TP :


---



In [1]:
####################################################################
# DEBUT DE CODE : SAE15 - TRAITER DES DONNEES                      #
#------------------------------------------------------------------#
# PARTIE 1 : IMPORTATIONS, INSTALLATIONS ET CONFIGURATIONS         #
# PARTIE 2 : RECUPERATION DES DONNEES VELIB EN OPENDATA            #
# PARTIE 3 : MISE SOUS FORME DE DATAFRAME                          #
# PARTIE 4 : ANALYSES STATISTIQUES                                 #
# PARTIE 5 : EXPORTATION DES MESURES STATISTIQUES POUR LE WEB      #
# PARTIE 6 : SPATIALISATION DES DONNEES, EXPORTATION POUR LE WEB   #
####################################################################

In [2]:
####################################################################
# PARTIE 1 : IMPORTATIONS, INSTALLATIONS ET CONFIGURATION          #
####################################################################
# modules natifs à importer
import datetime                 # pour la gestion des dates
import sys                      # pour l'accès au commandes système
from google.colab import drive  # pour l'accès au drive
import json                     # pour manipuler le format JSON
import pandas as pd             # pour extraire, calculer, publier
import geopandas as gpd         # pour la spatialisation des données
import matplotlib.pyplot as plt # pour les graphes
#-------------------------------------------------------------------
# modules externes à installer et à importer
# !pip install geopandas
!pip install contextily
import contextily as ctx        # pour l'utilisation de cartes géographiques
#-------------------------------------------------------------------
# montage du drive
drive.mount('/content/drive', force_remount=True)

#-------------------------------------------------------------------
# modules internes à importer
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/SAE15/tools')
import sae15_tools as tools
import sae15_spec as spec
####################################################################

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 44.8 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
####################################################################
# PARTIE 2 : RECUPERATION DES DONNEES VELIB EN OPENDATA            #
####################################################################
# récupération des données statiques (données d'information)
static_data = tools.loadVelibInformation()
static_data = tools.getVelibStations(static_data)
#-------------------------------------------------------------------
# récupération des données dynamiques (données de status)
dynamic_data = tools.loadVelibStatus()
dynamic_data = tools.getVelibStations(dynamic_data)
####################################################################

In [4]:
####################################################################
# PARTIE 3 : MISE SOUS FORME DE DATAFRAME                          #
####################################################################
# dataframe de l'information pour les stations
df_static = pd.DataFrame(static_data)

#-------------------------------------------------------------------
# dataframe du status pour les stations
df_dynamic = pd.DataFrame(dynamic_data)

#-------------------------------------------------------------------
# fusion des deux DataFrames en un seul (merge)
df = pd.merge(df_static, df_dynamic, on='station_id')

####################################################################

In [5]:
# Extracting mechanical and ebike data into separate DataFrames for analysis
mechanical_df = pd.DataFrame(df['num_bikes_available_types'].apply(lambda x: x[0]['mechanical']).tolist(), columns=['mechanical'])
ebike_df = pd.DataFrame(df['num_bikes_available_types'].apply(lambda x: x[1]['ebike']).tolist(), columns=['ebike'])

# Combine mechanical and ebike DataFrames into a single DataFrame
bikes = pd.concat([mechanical_df, ebike_df], axis=1)
bikes.head()

df['mechanical'] = bikes['mechanical']
df['ebike'] = bikes['ebike']
print("Column names:", df.columns)
df.head()

Column names: Index(['station_id', 'name', 'lat', 'lon', 'capacity', 'stationCode_x',
       'rental_methods', 'stationCode_y', 'num_bikes_available',
       'numBikesAvailable', 'num_bikes_available_types', 'num_docks_available',
       'numDocksAvailable', 'is_installed', 'is_returning', 'is_renting',
       'last_reported', 'mechanical', 'ebike'],
      dtype='object')


,station_id,name,lat,lon,capacity,stationCode_x,rental_methods,stationCode_y,num_bikes_available,numBikesAvailable,num_bikes_available_types,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported,mechanical,ebike
0,213688169,Benjamin Godard - Victor Hugo,48.865983,2.275725,35,16107,NaN,16107,6,6,"[{'mechanical': 3}, {'ebike': 3}]",29,29,1,1,1,1701262067,3,3
1,653222953,Mairie de Rosny-sous-Bois,48.871257,2.486581,30,31104,[CREDITCARD],31104,12,12,"[{'mechanical': 4}, {'ebike': 8}]",18,18,1,1,1,1701261966,4,8
2,17278902806,Rouget de L'isle - Watteau,48.778193,2.396302,20,44015,NaN,44015,7,7,"[{'mechanical': 1}, {'ebike': 6}]",13,13,1,1,1,1701262144,1,6
3,36255,Toudouze - Clauzel,48.879296,2.337360,21,9020,[CREDITCARD],9020,9,9,"[{'mechanical': 4}, {'ebike': 5}]",12,12,1,1,1,1701262075,4,5
4,37815204,Mairie du 12ème,48.840855,2.387555,30,12109,NaN,12109,5,5,"[{'mechanical': 3}, {'ebike': 2}]",25,25,1,1,1,1701261735,3,2


In [6]:
####################################################################
# PARTIE 4 : ANALYSES STATISTIQUES                                 #
####################################################################
# mesures statistiques sur la capacité des stations
stats_capacity = spec.stationStatistics(df, 'capacity')

#-------------------------------------------------------------------
# mesures statistiques sur les vélos disponibles
stats_bikes = spec.stationStatistics(df, 'num_bikes_available')

#-------------------------------------------------------------------
# mesures statistiques sur les stands disponibles
stats_stands = spec.stationStatistics(df, 'num_docks_available')

#-------------------------------------------------------------------
# mesures statistiques sur les vélos mécaniques (optionnel)
stats_mechanical = spec.stationStatistics(bikes, 'mechanical')

#-------------------------------------------------------------------
# mesures statistiques sur les vélos électriques (optionnel)
stats_ebike = spec.stationStatistics(bikes, 'ebike')
stats_ebike
#-------------------------------------------------------------------
# assemblage des toutes les statistiques dans un seul DataFrame
stats = pd.concat([stats_capacity, stats_bikes, stats_stands, stats_mechanical, stats_ebike], axis=0).T

#-------------------------------------------------------------------
# affichage dans la console du DataFrame des statistiques assemblées
display(stats)

####################################################################

,capacity,num_bikes_available,num_docks_available,mechanical,ebike
Mean,31.056501,10.643975,19.601770,6.703199,3.940776
Median,29.000000,6.000000,19.000000,3.000000,3.000000
Standard Deviation,12.005173,11.544394,12.913779,8.900854,4.488545
Min,0.000000,0.000000,0.000000,0.000000,0.000000
Max,74.000000,76.000000,64.000000,57.000000,37.000000
Count,1469.000000,1469.000000,1469.000000,1469.000000,1469.000000


In [7]:
####################################################################
# PARTIE 5 : EXPORTATION DES MESURES STATISTIQUES POUR LE WEB      #
####################################################################
# mise en forme du DataFrame des statistiques avant exportation
stats = stats.reset_index()
stats = stats.rename(columns={'index': 'statistics'})
# stats = stats.set_index('statistics')
display(stats)

#-------------------------------------------------------------------
# exportation du DataFrame des statistiques au format HTML
path = '/content/drive/MyDrive/Colab Notebooks/SAE15/web/data'
spec.exportStatistics(stats, path+'/stats.html')

####################################################################

,statistics,capacity,num_bikes_available,num_docks_available,mechanical,ebike
0,Mean,31.056501,10.643975,19.601770,6.703199,3.940776
1,Median,29.000000,6.000000,19.000000,3.000000,3.000000
2,Standard Deviation,12.005173,11.544394,12.913779,8.900854,4.488545
3,Min,0.000000,0.000000,0.000000,0.000000,0.000000
4,Max,74.000000,76.000000,64.000000,57.000000,37.000000
5,Count,1469.000000,1469.000000,1469.000000,1469.000000,1469.000000


In [8]:
####################################################################
# PARTIE 6 : SPATIALISATION DES DONNEES, EXPORTATION POUR LE WEB   #
####################################################################
# transformation du DataFrame des stations Vélib en GeoDataFrame
geo_stations = tools.exportToGeoDF(df)

#-------------------------------------------------------------------
# détermination de la dernière date de mise à jour des données
date = spec.getLatestDate(geo_stations)
path = '/content/drive/MyDrive/Colab Notebooks/SAE15/web/data'
#-------------------------------------------------------------------
# carte du taux de disponibilité des vélos avec date de mise à jour
# et export web de la carte
geo_stations['taux_velos'] = spec.availableBikesRate(geo_stations)
markersize=geo_stations['taux_velos']
spec.exportCityMap(geo_stations, 'taux_velos', markersize,tools.getRandomColorPallette() , 'carte du taux de disponibilité des vélos', date, path+'/carte_taux_velos.png')

#-------------------------------------------------------------------
# carte du taux de disponibilité des stands avec date de mise à jour
# et export web de la carte
geo_stations['taux_stands'] = spec.availableDocksRate(geo_stations)
markersize=geo_stations['taux_stands']
spec.exportCityMap(geo_stations, 'taux_stands', markersize,tools.getRandomColorPallette() , 'carte du taux de disponibilité des stands', date, path+'/carte_taux_stands.png')

#-------------------------------------------------------------------
# carte du taux de disponibilité des vélos mécaniques (optionnel)
# et export web de la carte
geo_stations['taux_mechanical'] = (bikes['mechanical'] * geo_stations['taux_velos'])/geo_stations['numBikesAvailable']
markersize=geo_stations['taux_mechanical']
spec.exportCityMap(geo_stations, 'taux_mechanical', markersize,tools.getRandomColorPallette() , 'carte du taux de disponibilité des vélos mécaniques', date, path+'/carte_taux_mechanical.png')

#-------------------------------------------------------------------
# carte du taux de disponibilité des vélos électriques (optionnel)
# et export web de la carte
geo_stations['taux_ebike'] = (bikes['ebike'] * geo_stations['taux_velos'])/geo_stations['numBikesAvailable']
markersize=geo_stations['taux_ebike']
spec.exportCityMap(geo_stations, 'taux_ebike', markersize,tools.getRandomColorPallette() , 'carte du taux de disponibilité des vélos électriques', date, path+'/carte_taux_ebike.png')

####################################################################

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
####################################################################
# FIN DE CODE : VERIFIER LA PUBLICATION SUR LE WEB                 #
####################################################################